# Performance analysis of Microgrids.jl

e.g. timing, profiling and typing issues

Run on Dell notebook with Intel Core i7-1165G7 @ 2.80GHz, *powered by the dock*

PH, June 2022

In [1]:
using Microgrids
using BenchmarkTools
using CSV, DataFrames

┌ Info: Precompiling Microgrids [bd581358-d3fa-499e-a26e-e70307242c03]
└ @ Base loading.jl:1423


In [2]:
"""load time series from CSV file"""
function load_ts()
    data = DataFrame(CSV.File("$(@__DIR__)/../examples/microgrid_with_PV_BT_DG/data/Ouessant_data_2016.csv"))
end

load_ts

## Microgrid data preparation

* **1.9 ms** to load CSV time series for 1 year
* otherwise, creating the `Microgrid` structure with all its components (`Photovoltaic`...) is negligible: 15 µs

In [3]:
@btime load_ts();

  1.894 ms (398 allocations: 1.23 MiB)


In [4]:
const my_data = load_ts();

In [5]:
function mg_create(data)
    # Simulation steps
    ntimestep = length(data.Load)

    # Components parameters
    # Project
    lifetime = 25
    discount_rate = 0.05
    timestep = 1
    # Load
    Pload = data."Load"[1:ntimestep]
    # Photovoltaic
    power_rated_PV = 4106.82251423571
    fPV = 1.
    IT = data."Ppv1k"[1:ntimestep] ./ 1000
    IS = 1.
    investiment_cost_PV = 1200.
    om_cost_PV = 20.  
    replacement_cost_PV = 1200.
    salvage_cost_PV = 1200.
    lifetime_PV = 25
    # Battery
    energy_initial = 0.
    energy_max = 6839.87944197573
    energy_min = 0
    power_min = -1.114*energy_max
    power_max = 1.002*energy_max
    loss = 0.05
    investiment_cost_BT = 350.
    om_cost_BT = 10.
    replacement_cost_BT = 350.
    salvage_cost_BT = 350.
    lifetime_BT = 15
    lifetime_thrpt = 3000
    # Diesel generator
    power_rated_DG = 1800.
    min_load_ratio = 0
    F0 = 0.0
    F1 = 0.240
    fuel_cost = 1.
    investiment_cost_DG = 400.
    om_cost_DG = 0.02
    replacement_cost_DG = 400.
    salvage_cost_DG = 400.
    lifetime_DG = 15000

    # Create microgrid components
    project = Project(lifetime, discount_rate, timestep)
    dieselgenerator = DieselGenerator(power_rated_DG, min_load_ratio, F0, F1, fuel_cost, investiment_cost_DG, om_cost_DG, replacement_cost_DG, salvage_cost_DG, lifetime_DG)
    photovoltaic = Photovoltaic(power_rated_PV, fPV, IT, IS, investiment_cost_PV, om_cost_PV, replacement_cost_PV, salvage_cost_PV, lifetime_PV)
    battery = Battery(energy_initial, energy_max, energy_min, power_min, power_max, loss, investiment_cost_BT, om_cost_BT, replacement_cost_BT, salvage_cost_BT, lifetime_BT, lifetime_thrpt)

    # Create microgrid
    microgrid = Microgrid(project, Pload, dieselgenerator, battery, [photovoltaic])
end

function mg_create()
    # Importing load and solar data
    data = load_ts()
    mg_create(data)
end

mg_create (generic function with 2 methods)

In [6]:
@btime mg_create();

  1.736 ms (415 allocations: 1.44 MiB)


In [7]:
@btime mg_create(my_data);

  9.748 μs (17 allocations: 206.19 KiB)


In [8]:
const my_mg = mg_create();

## Microgrid simulation

**7.7 ms** (7.7 MiB allocation) with an already created case description

In [9]:
function mg_sim(mg)
    # Run simulation
    results = simulate(mg)
end

function mg_sim()
    mg = mg_create()
    # Run simulation
    results = simulate(mg)
end

mg_sim (generic function with 2 methods)

In [10]:
@btime mg_sim(my_mg);

  4.628 ms (325736 allocations: 6.24 MiB)


In [24]:
@btime mg_sim(my_mg); # code with Any type

  7.697 ms (413499 allocations: 7.72 MiB)


### Timing of the 3 main steps of `simulate`

1. operation: **5.1 ms** (5.6 MiB alloc)
2. aggregation: **2.3 ms** (2.1 MiB alloc)
3. economics: 15 µs, negligible, *but still 12 KiB alloc*

for some reason, the sum (7.4 ms) is slighlty less than the global timing (7.7 ms). However, total allocation is consistent.

Conclusion: `operation(mg)` and `aggregation(mg, opervarstraj)` should be the focus of the performance optimization.

In [11]:
function simulate_time(mg)
    # Run the microgrid operation
    opervarstraj = @btime operation($mg)

    # Aggregate the operation variables
    opervarsaggr = @btime aggregation($mg, $opervarstraj)

    # Eval the microgrid costs
    costs = @btime economics($mg, $opervarsaggr)

    return (opervarstraj = opervarstraj, opervarsaggr = opervarsaggr, costs = costs)
end

simulate_time (generic function with 1 method)

In [12]:
simulate_time(my_mg);

  2.261 ms (138368 allocations: 3.25 MiB)
  1.864 ms (187241 allocations: 2.99 MiB)
  7.275 μs (127 allocations: 5.22 KiB)


In [36]:
simulate_time(my_mg); # code with Any type

  5.115 ms (285064 allocations: 5.62 MiB)
  2.267 ms (127928 allocations: 2.09 MiB)
  15.143 μs (507 allocations: 12.38 KiB)


## Understanding type issues

Observation: typing in `operation(mg)` is terrible since all variables are typed as `Any`. In particular, the call to the dispatch:

```julia
outputs = dispatch(power_net_load_requested[i], Pbatt_cmax[i], Pbatt_dmax[i], mg.dieselgenerator.power_rated)
```

becomes:

```
│   %71  = Base.getindex(power_net_load_requested, i)::Any
│   %72  = Base.getindex(Pbatt_cmax, i)::Any
│   %73  = Base.getindex(Pbatt_dmax, i)::Any
│   %74  = Base.getproperty(mg, :dieselgenerator)::DieselGenerator
│   %75  = Base.getproperty(%74, :power_rated)::Any
│   %76  = Microgrids.dispatch(%71, %72, %73, %75)::NTuple{5, Any}
```


In [13]:
@code_warntype operation(my_mg);

MethodInstance for Microgrids.operation(::Microgrid)
  from operation(mg::Microgrid) in Microgrids at /home/pierre/Travail/31 Programmes divers/10 MicroGrid/Microgrid.jl/src/operation.jl:7
Arguments
  #self#::Core.Const(Microgrids.operation)
  mg::Microgrid
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  #1::Microgrids.var"#1#2"
  opervarstraj::OperVarsTraj
  Pshed::Vector{Float64}
  Pcurt::Vector{Float64}
  Pbatt_cmax::Vector{Float64}
  Pbatt_dmax::Vector{Float64}
  Pbatt::Vector{Float64}
  Ebatt::Vector{Float64}
  Pgen::Vector{Float64}
  power_net_load::Vector{Float64}
  T::Type{Float64}
  stepsnumber::Int64
  power_net_load_requested::Any
  total_renewables_production::Any
  renewables_production::Any
  @_19::Int64
  i::Int64
  Pb_emax::Float64
  Pb_emin::Float64
Body::OperVarsTraj
1 ─        Core.NewvarNode(:(opervarstraj))
│          (#1 = %new(Microgrids.:(var"#1#2")))
│   %3   = #1::Core.Const(Microgrids.var"#1#2"())
│   %4   = Base.getproperty(mg, :nondispatchables)::Vector

In [45]:
@code_warntype operation(my_mg); # code with Any type

MethodInstance for Microgrids.operation(::Microgrid)
  from operation(mg::Microgrid) in Microgrids at /home/pierre/Travail/31 Programmes divers/10 MicroGrid/Microgrid.jl/src/operation.jl:7
Arguments
  #self#::Core.Const(Microgrids.operation)
  mg::Microgrid
Locals
  @_3::Any
  #1::Microgrids.var"#1#2"
  opervarstraj::OperVarsTraj
  Pshed::Any
  Pcurt::Any
  Pbatt_cmax::Any
  Pbatt_dmax::Any
  Pbatt::Any
  Ebatt::Any
  Pgen::Any
  power_net_load::Any
  stepsnumber::Any
  power_net_load_requested::Any
  total_renewables_production::Any
  renewables_production::Any
  @_18::Int64
  i::Any
  Pb_emax::Any
  Pb_emin::Any
Body::OperVarsTraj
1 ─        Core.NewvarNode(:(opervarstraj))
│          (#1 = %new(Microgrids.:(var"#1#2")))
│   %3   = #1::Core.Const(Microgrids.var"#1#2"())
│   %4   = Base.getproperty(mg, :nondispatchables)::Vector{NonDispatchables}
│   %5   = Base.Generator(%3, %4)::Base.Generator{Vector{NonDispatchables}, Microgrids.var"#1#2"}
│   %6   = Microgrids.collect(%5)::Vector
